In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Data Cleaning

In [52]:
excel = pd.ExcelFile('HainanClean_New.xlsx')
hainan = excel.parse("fulldf")
hainan
hainan.columns = hainan.columns.str.replace('  ', '_')
hainan.columns = hainan.columns.str.replace(' ', '_')
hainan.columns = hainan.columns.str.replace('(', '')
hainan.columns = hainan.columns.str.replace('（', '')
hainan.columns = hainan.columns.str.replace(')', '')

d = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6,
     'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
hainan.Month = hainan.Month.map(d)
hainan.BioCNG_Produced_Nm3 = hainan.BioCNG_Produced_m3.shift(-15)
hainan.drop(hainan.tail(15).index,inplace=True)
hainan = hainan[np.isfinite(hainan['Month'])]
hainan['BioCNG_cumsum'] = hainan.BioCNG_Produced_m3.cumsum()
hainan.columns
hainan = hainan[np.isfinite(hainan['Lemon_waste_t'])]
hainan = hainan[np.isfinite(hainan['Percolate_t'])]
hainan = hainan.replace(' ',0)
hainan = hainan.replace('',0)
hainan = hainan.replace('  ',0)
hainan = hainan.drop(['Year', 'Month', 'Day', 'Month_#', 'Day_#', 'Raw_Biogas_Produced_m3', 'BioCNG_Sold_m3', 'Vehicle_use_m3',
       'Liquid_Fertilizer_Produced_t', 'Solid_fertilizer_produced_t',
       'Wastewater_flow_to_WWTP_unit?', 'Solid_residues_kg','50%_NaOH/kg', 'FeCl2/kg', 'PAM/kg',
       'Defoamer/kg', 'Project_electricity_use/kWh',
       'Office_space_electricity_use/kWh', 'Water/m3', 'Diesel/L'], axis=1)
sumcum = hainan['BioCNG_cumsum']
hainan.drop(labels=['BioCNG_cumsum'], axis=1,inplace = True)
hainan.insert(0, 'BioCNG_cumsum', sumcum)
hainan.head()

,BioCNG_cumsum,BioCNG_Produced_m3,Pig_Manure_t,Cassava_t,Fish_waste_water_t,Kitchen_food_waste_t,Municipal_fecal_residue_t,Tea_waste_t,Chicken_litter_t,Bagasse_feed_t,Alcohol_waste_t,Chinese_medicine_waste_t,Energy_grass_t,Banana_fruit_shafts_t,Lemon_waste_t,Percolate_t,Other_waste_t
0,2024.0,2024.0,6.82,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,26.44
1,4933.0,2909.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,28.92
2,7953.0,3020.0,6.66,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,26.90
3,10995.0,3042.0,12.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,13774.0,2779.0,9.54,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


# Preliminary Work

In [53]:
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
print('train data len:',len(hainan_train))
print('test data len:',len(hainan_test))

train data len: 1098
test data len: 275


In [54]:
#ols
hainan_ols = ols("BioCNG_cumsum ~ Pig_Manure_t + Cassava_t + Fish_waste_water_t +\
                    Kitchen_food_waste_t + Municipal_fecal_residue_t + Tea_waste_t +\
                    Chicken_litter_t + Bagasse_feed_t + Alcohol_waste_t +\
                    Chinese_medicine_waste_t + Energy_grass_t + Banana_fruit_shafts_t +\
                    Lemon_waste_t + Percolate_t + Other_waste_t", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          BioCNG_cumsum   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     395.0
Date:                Fri, 16 Mar 2018   Prob (F-statistic):               0.00
Time:                        17:53:44   Log-Likelihood:                -16457.
No. Observations:                1098   AIC:                         3.295e+04
Df Residuals:                    1082   BIC:                         3.303e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                   1.38e+06   5.01e+04     27.521      0.000    1.28e+06    1.48e+06
Pig_Manure_t               3.285e+04   2593.569     12.664      0.000    2.78e+04    3.79e+04
Cassava_t                    92.6948   1017.263      0.091      0.927   -1903.337    2088.726
Fish_waste_water_t         3797.6252   4668.275      0.813      0.416   -5362.273     1.3e+04
Kitchen_food_waste_t       2.929e+04   2507.773     11.679      0.000    2.44e+04    3.42e+04
Municipal_fecal_residue_t  6521.0966   1025.493      6.359      0.000    4508.917    8533.276
Tea_waste_t                2.915e+04   8667.214      3.363      0.001    1.21e+04    4.62e+04
Chicken_litter_t           1.719e+04   3921.882      4.382      0.000    9490.456    2.49e+04
Bagasse_feed_t            -1.576e+04   1696.223     -9.292      0.000   -1.91e+04   -1.24e+04
Alcohol_waste_t            3.515e+04   6576.510      5.345      0.000    2.22e+04    4.81e+04
Chinese_medicine_waste_t   3.598e+04   2.29e+04      1.570      0.117   -8982.518    8.09e+04
Energy_grass_t              3.22e+04   1.69e+04      1.905      0.057    -966.672    6.54e+04
Banana_fruit_shafts_t     -3003.3583   6434.448     -0.467      0.641   -1.56e+04    9622.052
Lemon_waste_t             -8229.3934   2.07e+04     -0.398      0.691   -4.88e+04    3.23e+04
Percolate_t                2295.3850    281.575      8.152      0.000    1742.889    2847.881
Other_waste_t              5970.2576   2431.348      2.456      0.014    1199.567    1.07e+04
==============================================================================
Omnibus:                       35.445   Durbin-Watson:                   2.060
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.625
Skew:                           0.285   Prob(JB):                     1.65e-06
Kurtosis:                       2.493   Cond. No.                         362.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#ols
hainan_ols = ols("Solid_fertilizer_produced_t ~ Pig_Manure_t + Cassava_t + Fish_waste_water_t +\
                    Kitchen_food_waste_t + Municipal_fecal_residue_t + Tea_waste_t +\
                    Chicken_litter_t + Bagasse_feed_t + Alcohol_waste_t +\
                    Chinese_medicine_waste_t + Energy_grass_t + Banana_fruit_shafts_t +\
                    Lemon_waste_t + Percolate_t + Other_waste_t", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary

From the ols results based on three different dependent variable, we can see that only the first one (cumulative sum of BioCNG) has a relatively high value for R-squared. So We decided to focus on the first one.

In [55]:
# Predict on the training data
X_train = hainan_train[['Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t']]
y_train = hainan_train.BioCNG_cumsum
y_pred_train = hainan_ols.predict(X_train)

# Compute the root-mean-square of training data
rms_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print('Training error:',rms_train/sum(y_train))

# Predict on the test data
X_test = hainan_test[['Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t']]
y_test = hainan_test.BioCNG_cumsum
y_pred_test = hainan_ols.predict(X_test)

# Compute the root-mean-square of test data
rms_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print('Testing error:',rms_test/sum(y_test))

hainan_train.head()

Training error: 0.000268314680063
Testing error: 0.00118491058929


,BioCNG_cumsum,BioCNG_Produced_m3,Pig_Manure_t,Cassava_t,Fish_waste_water_t,Kitchen_food_waste_t,Municipal_fecal_residue_t,Tea_waste_t,Chicken_litter_t,Bagasse_feed_t,Alcohol_waste_t,Chinese_medicine_waste_t,Energy_grass_t,Banana_fruit_shafts_t,Lemon_waste_t,Percolate_t,Other_waste_t
1138,4714893.00,10485.0,13.08,0.00,0.0,36.06,58.82,8.14,0.00,35.0,0.0,0.00,0.0,0.0,0.00,309.0,0.0
1310,6554771.47,12318.0,24.34,0.00,0.0,54.20,111.74,0.00,26.02,0.0,0.0,0.00,0.0,0.0,5.22,421.0,0.0
1062,4081906.00,8563.0,36.24,0.00,0.0,27.80,59.96,0.00,13.72,0.0,0.0,5.82,0.0,0.0,0.00,200.0,0.0
673,2216048.00,6508.0,19.22,52.04,0.0,0.00,0.00,0.00,0.00,13.5,0.0,0.00,0.0,0.0,0.00,0.0,0.0
1238,5841568.00,11795.0,11.50,0.00,0.0,71.76,196.64,0.00,25.76,0.0,0.0,0.00,0.0,0.0,0.00,387.0,0.0


# Training Algorithms

1. Logistic regression
2. SVM
3. Perceptron
4. kNN
5. Random Forest
6. xgBoost

In [33]:
#logistic regression
hainan['BioCNG_cumsum'] = pd.cut(hainan['BioCNG_cumsum'], bins=3, labels=[0, 1, 2])
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
y_train = hainan_train.BioCNG_cumsum
y_test = hainan_test.BioCNG_cumsum
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of logistic regression classifier on train set: 0.52
Accuracy of logistic regression classifier on test set: 0.52
[[143   2   0]
 [ 89   0   0]
 [ 37   4   0]]


In [34]:
# SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)  
y_pred_test = clf.predict(X_test)
print('Accuracy of SVM on train set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of SVM on test set: {:.2f}'.format(clf.score(X_test, y_test)))

Accuracy of SVM on train set: 0.85
Accuracy of SVM on test set: 0.51


In [35]:
# kNN
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=4)
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(hainan[['BioCNG_cumsum', 'Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t']])
hainan_normalized = pd.DataFrame(np_scaled)
hainan_normalized_train, hainan_normalized_test = train_test_split(hainan_normalized, test_size=0.2)
hainan_normalized
Xnor_train = hainan_normalized_train.iloc[:, 1:16]    
ynor_train = hainan_normalized_train[0]
Xnor_test = hainan_normalized_test.iloc[:, 1:16]
ynor_test = hainan_normalized_test[0]
knn.fit(Xnor_train, ynor_train)
y_pred_test = knn.predict(Xnor_test)
print('Accuracy of kNN on train set: {:.2f}'.format(knn.score(Xnor_train, ynor_train)))
print('Accuracy of kNN on test set: {:.2f}'.format(knn.score(Xnor_test, ynor_test)))


Accuracy of kNN on train set: 0.89
Accuracy of kNN on test set: 0.83


In [36]:
# xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred_test = xgb.predict(X_test)
print('Accuracy of XGboost on train set: {:.2f}'.format(xgb.score(X_train, y_train)))
print('Accuracy of XGboost on test set: {:.2f}'.format(xgb.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of XGboost on train set: 0.62
Accuracy of XGboost on test set: 0.50
[[132  13   0]
 [ 83   6   0]
 [ 38   3   0]]


In [37]:
# random forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 1000)
random_forest.fit(X_train, y_train)
print('Accuracy of Random Forest on train set: {:.2f}'.format(random_forest.score(X_train, y_train)))
print('Accuracy of Random Forest on test set: {:.2f}'.format(random_forest.score(X_test, y_test)))

Accuracy of Random Forest on train set: 0.97
Accuracy of Random Forest on test set: 0.48


From the accuracy results of the above algorithms training, we can see that except for kNN, all of them have relatively good results for accuracy around 0.5.

# Trying with 1/x

In [56]:
for col in hainan.columns[2:]:
    hainan['1/'+col] = 1/(hainan[col])
hainan.head()
hainan.columns

Index(['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t'],
      dtype='object')

In [57]:
hainan.replace(float('inf'), float(0), inplace = True);
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
print('train data len:',len(hainan_train))
print('test data len:',len(hainan_test))
hainan.head()

train data len: 1098
test data len: 275


,BioCNG_cumsum,BioCNG_Produced_m3,Pig_Manure_t,Cassava_t,Fish_waste_water_t,Kitchen_food_waste_t,Municipal_fecal_residue_t,Tea_waste_t,Chicken_litter_t,Bagasse_feed_t,...,1/Tea_waste_t,1/Chicken_litter_t,1/Bagasse_feed_t,1/Alcohol_waste_t,1/Chinese_medicine_waste_t,1/Energy_grass_t,1/Banana_fruit_shafts_t,1/Lemon_waste_t,1/Percolate_t,1/Other_waste_t
0,2024.0,2024.0,6.82,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.037821
1,4933.0,2909.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.034578
2,7953.0,3020.0,6.66,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.037175
3,10995.0,3042.0,12.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,13774.0,2779.0,9.54,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [58]:
# Predict on the training data
X_train = hainan_train[['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t']]
y_train = hainan_train.BioCNG_cumsum
y_pred_train = hainan_ols.predict(X_train)

# Compute the root-mean-square of training data
rms_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print('Training error:',rms_train/sum(y_train))

# Predict on the test data
X_test = hainan_test[['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t']]
y_test = hainan_test.BioCNG_cumsum
y_pred_test = hainan_ols.predict(X_test)

# Compute the root-mean-square of test data
rms_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print('Testing error:',rms_test/sum(y_test))

hainan_train.head()

Training error: 0.000273629873438
Testing error: 0.00109429590844


,BioCNG_cumsum,BioCNG_Produced_m3,Pig_Manure_t,Cassava_t,Fish_waste_water_t,Kitchen_food_waste_t,Municipal_fecal_residue_t,Tea_waste_t,Chicken_litter_t,Bagasse_feed_t,...,1/Tea_waste_t,1/Chicken_litter_t,1/Bagasse_feed_t,1/Alcohol_waste_t,1/Chinese_medicine_waste_t,1/Energy_grass_t,1/Banana_fruit_shafts_t,1/Lemon_waste_t,1/Percolate_t,1/Other_waste_t
956,3180562.00,8130.0,39.00,25.0,0.00,4.48,26.24,0.0,0.00,0.00,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.003846,0.0
259,656481.00,5397.0,11.60,40.0,0.00,0.00,0.00,0.0,0.00,4.00,...,0.0,0.000000,0.250000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1337,6904337.47,14118.0,30.96,0.0,6.68,126.64,91.88,0.0,47.64,20.25,...,0.0,0.020991,0.049383,0.0,0.0,0.0,0.0,0.0,0.007143,0.0
105,244648.00,3191.0,10.24,0.0,0.00,0.00,0.00,0.0,0.00,40.00,...,0.0,0.000000,0.025000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
637,2025532.00,10934.0,19.66,126.0,4.70,0.00,0.00,0.0,0.00,3.75,...,0.0,0.000000,0.266667,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [59]:
#logistic regression
hainan['BioCNG_cumsum'] = pd.cut(hainan['BioCNG_cumsum'], bins=3, labels=[0, 1, 2])
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
y_train = hainan_train.BioCNG_cumsum
y_test = hainan_test.BioCNG_cumsum
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of logistic regression classifier on train set: 0.52
Accuracy of logistic regression classifier on test set: 0.53
[[146   0   0]
 [ 86   0   0]
 [ 43   0   0]]


In [60]:
# SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)  
y_pred_test = clf.predict(X_test)
print('Accuracy of SVM on train set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of SVM on test set: {:.2f}'.format(clf.score(X_test, y_test)))

Accuracy of SVM on train set: 1.00
Accuracy of SVM on test set: 0.53


In [61]:
# perceptron
from sklearn.linear_model import perceptron
net = perceptron.Perceptron(n_iter=100, verbose=0, random_state=None, fit_intercept=True, eta0=0.002)
net.fit(X_train, y_train)
y_pred_test = net.predict(X_test)
print('Accuracy of perceptron on train set: {:.2f}'.format(net.score(X_train, y_train)))
print('Accuracy of perceptron on test set: {:.2f}'.format(net.score(X_test, y_test)))

Accuracy of perceptron on train set: 0.52
Accuracy of perceptron on test set: 0.53


/Users/anna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [62]:
# kNN
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=4)
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(hainan[['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t']])
hainan_normalized = pd.DataFrame(np_scaled)
hainan_normalized_train, hainan_normalized_test = train_test_split(hainan_normalized, test_size=0.2)
hainan_normalized
Xnor_train = hainan_normalized_train.iloc[:, 1:16]    
ynor_train = hainan_normalized_train[0]
Xnor_test = hainan_normalized_test.iloc[:, 1:16]
ynor_test = hainan_normalized_test[0]
knn.fit(Xnor_train, ynor_train)
y_pred_test = knn.predict(Xnor_test)
print('Accuracy of kNN on train set: {:.2f}'.format(knn.score(Xnor_train, ynor_train)))
print('Accuracy of kNN on test set: {:.2f}'.format(knn.score(Xnor_test, ynor_test)))


Accuracy of kNN on train set: 0.90
Accuracy of kNN on test set: 0.77


In [63]:
# random forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 1000)
random_forest.fit(X_train, y_train)
print('Accuracy of Random Forest on train set: {:.2f}'.format(random_forest.score(X_train, y_train)))
print('Accuracy of Random Forest on test set: {:.2f}'.format(random_forest.score(X_test, y_test)))

Accuracy of Random Forest on train set: 1.00
Accuracy of Random Forest on test set: 0.46


# Trying with x^2

In [64]:
for col in hainan.columns[2:]:
    hainan[col+"**2"] = (hainan[col])**2
hainan.head()

,BioCNG_cumsum,BioCNG_Produced_m3,Pig_Manure_t,Cassava_t,Fish_waste_water_t,Kitchen_food_waste_t,Municipal_fecal_residue_t,Tea_waste_t,Chicken_litter_t,Bagasse_feed_t,...,1/Tea_waste_t**2,1/Chicken_litter_t**2,1/Bagasse_feed_t**2,1/Alcohol_waste_t**2,1/Chinese_medicine_waste_t**2,1/Energy_grass_t**2,1/Banana_fruit_shafts_t**2,1/Lemon_waste_t**2,1/Percolate_t**2,1/Other_waste_t**2
0,0,2024.0,6.82,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,0.0,0.004444,0.0,0.0,0.0,0.0,0.0,0.0,0.001430
1,0,2909.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,0.0,0.004444,0.0,0.0,0.0,0.0,0.0,0.0,0.001196
2,0,3020.0,6.66,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.0,0.0,0.004444,0.0,0.0,0.0,0.0,0.0,0.0,0.001382
3,0,3042.0,12.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0,2779.0,9.54,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.012346,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [76]:
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
print('train data len:',len(hainan_train))
print('test data len:',len(hainan_test))
hainan.head()
hainan.columns

train data len: 1098
test data len: 275


Index(['BioCNG_cumsum', 'BioCNG_Produced_m3', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t', 'Pig_Manure_t**2', 'Cassava_t**2',
       'Fish_waste_water_t**2', 'Kitchen_food_waste_t**2',
       'Municipal_fecal_residue_t**2', 'Tea_waste_t**2', 'Chicken_litter_t**2',
       'Bagasse_feed_t**2', 'Alcohol_waste_t**2',
       'Chinese_medicine_waste_t**2', 

In [70]:
# Predict on the training data
X_train = hainan_train[['Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t', 'Pig_Manure_t**2', 'Cassava_t**2',
       'Fish_waste_water_t**2', 'Kitchen_food_waste_t**2',
       'Municipal_fecal_residue_t**2', 'Tea_waste_t**2', 'Chicken_litter_t**2',
       'Bagasse_feed_t**2', 'Alcohol_waste_t**2',
       'Chinese_medicine_waste_t**2', 'Energy_grass_t**2',
       'Banana_fruit_shafts_t**2', 'Lemon_waste_t**2', 'Percolate_t**2',
       'Other_waste_t**2', '1/Pig_Manure_t**2', '1/Cassava_t**2',
       '1/Fish_waste_water_t**2', '1/Kitchen_food_waste_t**2',
       '1/Municipal_fecal_residue_t**2', '1/Tea_waste_t**2',
       '1/Chicken_litter_t**2', '1/Bagasse_feed_t**2', '1/Alcohol_waste_t**2',
       '1/Chinese_medicine_waste_t**2', '1/Energy_grass_t**2',
       '1/Banana_fruit_shafts_t**2', '1/Lemon_waste_t**2', '1/Percolate_t**2',
       '1/Other_waste_t**2']]
y_train = hainan_train.BioCNG_cumsum
y_pred_train = hainan_ols.predict(X_train)

# Predict on the test data
X_test = hainan_test[['Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t', 'Pig_Manure_t**2', 'Cassava_t**2',
       'Fish_waste_water_t**2', 'Kitchen_food_waste_t**2',
       'Municipal_fecal_residue_t**2', 'Tea_waste_t**2', 'Chicken_litter_t**2',
       'Bagasse_feed_t**2', 'Alcohol_waste_t**2',
       'Chinese_medicine_waste_t**2', 'Energy_grass_t**2',
       'Banana_fruit_shafts_t**2', 'Lemon_waste_t**2', 'Percolate_t**2',
       'Other_waste_t**2', '1/Pig_Manure_t**2', '1/Cassava_t**2',
       '1/Fish_waste_water_t**2', '1/Kitchen_food_waste_t**2',
       '1/Municipal_fecal_residue_t**2', '1/Tea_waste_t**2',
       '1/Chicken_litter_t**2', '1/Bagasse_feed_t**2', '1/Alcohol_waste_t**2',
       '1/Chinese_medicine_waste_t**2', '1/Energy_grass_t**2',
       '1/Banana_fruit_shafts_t**2', '1/Lemon_waste_t**2', '1/Percolate_t**2',
       '1/Other_waste_t**2']]
y_test = hainan_test.BioCNG_cumsum
y_pred_test = hainan_ols.predict(X_test)


In [71]:
#logistic regression
hainan['BioCNG_cumsum'] = pd.cut(hainan['BioCNG_cumsum'], bins=3, labels=[0, 1, 2])
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
y_train = hainan_train.BioCNG_cumsum
y_test = hainan_test.BioCNG_cumsum
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of logistic regression classifier on train set: 0.53
Accuracy of logistic regression classifier on test set: 0.51
[[135   4   0]
 [ 83   5   1]
 [ 47   0   0]]


In [72]:
# SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)  
y_pred_test = clf.predict(X_test)
print('Accuracy of SVM on train set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of SVM on test set: {:.2f}'.format(clf.score(X_test, y_test)))

Accuracy of SVM on train set: 0.96
Accuracy of SVM on test set: 0.50


In [73]:
# perceptron
from sklearn.linear_model import perceptron
net = perceptron.Perceptron(n_iter=100, verbose=0, random_state=None, fit_intercept=True, eta0=0.002)
net.fit(X_train, y_train)
y_pred_test = net.predict(X_test)
print('Accuracy of perceptron on train set: {:.2f}'.format(net.score(X_train, y_train)))
print('Accuracy of perceptron on test set: {:.2f}'.format(net.score(X_test, y_test)))

Accuracy of perceptron on train set: 0.45
Accuracy of perceptron on test set: 0.41


/Users/anna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [77]:
# kNN
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=4)
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(hainan[['BioCNG_cumsum', 'Pig_Manure_t', 'Cassava_t',
       'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '1/Pig_Manure_t', '1/Cassava_t',
       '1/Fish_waste_water_t', '1/Kitchen_food_waste_t',
       '1/Municipal_fecal_residue_t', '1/Tea_waste_t', '1/Chicken_litter_t',
       '1/Bagasse_feed_t', '1/Alcohol_waste_t', '1/Chinese_medicine_waste_t',
       '1/Energy_grass_t', '1/Banana_fruit_shafts_t', '1/Lemon_waste_t',
       '1/Percolate_t', '1/Other_waste_t', 'Pig_Manure_t**2', 'Cassava_t**2',
       'Fish_waste_water_t**2', 'Kitchen_food_waste_t**2',
       'Municipal_fecal_residue_t**2', 'Tea_waste_t**2', 'Chicken_litter_t**2',
       'Bagasse_feed_t**2', 'Alcohol_waste_t**2',
       'Chinese_medicine_waste_t**2', 'Energy_grass_t**2',
       'Banana_fruit_shafts_t**2', 'Lemon_waste_t**2', 'Percolate_t**2',
       'Other_waste_t**2', '1/Pig_Manure_t**2', '1/Cassava_t**2',
       '1/Fish_waste_water_t**2', '1/Kitchen_food_waste_t**2',
       '1/Municipal_fecal_residue_t**2', '1/Tea_waste_t**2',
       '1/Chicken_litter_t**2', '1/Bagasse_feed_t**2', '1/Alcohol_waste_t**2',
       '1/Chinese_medicine_waste_t**2', '1/Energy_grass_t**2',
       '1/Banana_fruit_shafts_t**2', '1/Lemon_waste_t**2', '1/Percolate_t**2',
       '1/Other_waste_t**2']])
hainan_normalized = pd.DataFrame(np_scaled)
hainan_normalized_train, hainan_normalized_test = train_test_split(hainan_normalized, test_size=0.2)
hainan_normalized
Xnor_train = hainan_normalized_train.iloc[:, 1:60]    
ynor_train = hainan_normalized_train[0]
Xnor_test = hainan_normalized_test.iloc[:, 1:60]
ynor_test = hainan_normalized_test[0]
knn.fit(Xnor_train, ynor_train)
y_pred_test = knn.predict(Xnor_test)
print('Accuracy of kNN on train set: {:.2f}'.format(knn.score(Xnor_train, ynor_train)))
print('Accuracy of kNN on test set: {:.2f}'.format(knn.score(Xnor_test, ynor_test)))


Accuracy of kNN on train set: 0.87
Accuracy of kNN on test set: 0.88


In [78]:
# random forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 1000)
random_forest.fit(X_train, y_train)
print('Accuracy of Random Forest on train set: {:.2f}'.format(random_forest.score(X_train, y_train)))
print('Accuracy of Random Forest on test set: {:.2f}'.format(random_forest.score(X_test, y_test)))

Accuracy of Random Forest on train set: 0.98
Accuracy of Random Forest on test set: 0.51


# Trying with logx